# Giving model time to think

Just like humans, LLMs also need time to think of a solution. Humans generally tend to think of solution by going through steps. If there is mathematical calculation involved, humans solve it by calculating the parts of the equation first and then substituting the value in final equation to get final answers. Similarly LLM may also get confused we final answer is asked directly. In this notebook we show how the LLM answers can improve in calculation accuracy by asking the model to sove in steps or give it the steps to solve.

We will first import required libraries and load the API key.

In [ ]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')

We create a helper function which is useful to get the completion of the input prompt using an LLM (GPT-3.5-turbo in this case).

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

## Without giving model time to think

We first demonstrate CAGR calculation by just asking LLM for the final answer. We observe that it does have the correct formula to solve and does correct substitution but misses calculating the correct answer.

In [ ]:
prompt = f"""
Calculate CAGR for start value: 100, ending value: 150 and for 2 years
"""
response = get_completion(prompt)
print(response)

The formula for CAGR (Compound Annual Growth Rate) is:

CAGR = (Ending Value / Start Value)^(1 / Number of Years) - 1

Plugging in the values given:

CAGR = (150 / 100)^(1 / 2) - 1

CAGR = 1.22 - 1

CAGR = 0.22 or 22%

Therefore, the CAGR for a start value of 100, ending value of 150, and a 2-year period is 22%.

## Tactics 1: Providing Steps

In this prompt, we give exact steps to calculate CAGR and ask the LLM to calculte one step at a time. We observe that we get more accurate solution in this case compared to when we ask it to directly calculate final answer.

In [ ]:
prompt = f"""
Calculate CAGR for start value: 100, ending value: 150 and for 2 years

This is how CAGR can be calculated:
step 1: calculate 1 over period
step 2: calculate end value over start value
step 3: raise the value you got from step 2 to the power of the value you got from step 1
step 4: subtract 1 from whatever you got from step 3
"""
response = get_completion(prompt)
print(response)

For the given values, the calculation would be:

Step 1: 1/2 = 0.5 

Step 2: 150/100 = 1.5 

Step 3: 1.5^0.5 = 1.2247 

Step 4: 1.2247 - 1 = 0.2247

Therefore, the CAGR for a start value of 100, ending value of 150 and for 2 years is 22.47%.

## Tactics 2: Ask model to work out solution

Sometimes it might not make sense to provide the steps ourselves as that might defeat the whole purpose of having an LLM perform the calcualtion. In this example, we specifically prompt the LLM not to calculate the CAGR in one go but to break down the fundamental math sub-problems and perform the calculation in steps. We observe that we get answer with similar accuracy as when we explicitly gave steps ourselves.

In [ ]:
prompt = f"""
Calculate CAGR for start value: 100, ending value: 150 and for 2 years
Dont calculate the whole equation in one step. \
Break down the calculation and perform smallest atomic calculations like \
division, multiplication, etc first.
"""
response = get_completion(prompt)
print(response)

The formula for CAGR (Compound Annual Growth Rate) is:

CAGR = (Ending Value / Start Value)^(1/Number of Years) - 1

In this case, the start value is 100, the ending value is 150, and the number of years is 2. 

First, we need to calculate the growth factor:

Growth Factor = Ending Value / Start Value = 150 / 100 = 1.5

Next, we need to calculate the exponent:

Exponent = 1 / Number of Years = 1 / 2 = 0.5

Now we can calculate the CAGR:

CAGR = Growth Factor^(Exponent) - 1 = 1.5^(0.5) - 1 = 0.2247 or 22.47%

Therefore, the CAGR for a start value of 100, ending value of 150, and 2 years is 22.47%.